# Pose estimation


In [ ]:
import os
os.listdir("./")

In [ ]:

import numpy as np
data = np.load("./obj_pose_3.npz")
for k,v in data.items():
    print(k,"shape",v.shape)
    
obs = data["arr_3"]

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from ipywidgets import Button, Layout
from ipywidgets import interact

fig,ax = plt.subplots(1,1)
ax.set_axis_off()
plt_h = ax.imshow(obs[0],vmin=0,vmax=255,interpolation="bicubic")

def update(i=0):
    image = obs[i]
    plt_h.set_data(image)
    fig.canvas.draw()
    
tmp = interact(update, i=(0,obs.shape[0]-1),layout=Layout(width='100%'))


In [ ]:
from PIL import Image, ImageEnhance
from skimage import measure
i = 55

fig,ax = plt.subplots(1,2)
for a in ax:
    a.set_axis_off()
plt_h = ax[0].imshow(obs[0],vmin=0,vmax=255,interpolation="None")
plt_c = ax[1].imshow(obs[0],vmin=0,vmax=255,interpolation="None")

def get_mask(image, c, t):
    image = img = Image.fromarray(image)
    converter = ImageEnhance.Color(image)
    image = converter.enhance(c)
    image = np.array(image)
    
    mask = image[:,:,1] < t
    return image, mask

def update(c=25, t=234):
    image = obs[i].copy()
    cimage, mask = get_mask(image,c,t)
    
    image[np.logical_not(mask)] = 0
    plt_h.set_data(image)
    plt_c.set_data(cimage)
    fig.canvas.draw()
    
tmp = interact(update, c=(0,50),t=(0,255),layout=Layout(width='100%'))

In [ ]:
i = 55
c = 25
t = 234

image = obs[i].copy()
_,mask = get_mask(image,c,t)
all_labels = measure.label(mask)
plt.imshow(all_labels == 3)
plt.show()


In [ ]:
from gym_grasping.envs.grasping_env import GraspingEnv
plt.rcParams['figure.figsize'] = [8, 5]

env = GraspingEnv(env_name='block', renderer='tiny', max_steps=None,
                  initial_pose='fixed')

env.robot.set_navigate_world()
env.robot.use_simulation = False
assert env.robot.gripper.act_type == 'continuous'

In [ ]:
env.reset()
pos_orig, orn_orig = env.p.getBasePositionAndOrientation(env._task.objects[0])

policy = env._task.get_policy(mode='random')(env)

env_done = False
for iter in range(1):
    if env_done:
        env.reset()
    action = policy.act(env)
    state, reward, env_done, info = env.step(action)
    if env_done and reward == 1:
        print("success!")
    


In [ ]:
from IPython.display import display
from ipywidgets import interactive
from math import pi
from collections import OrderedDict

render = env.render()

fig, ax = plt.subplots(1,3)
for x in ax: x.set_axis_off()
fig.suptitle("Default Views (different aspect ratio)")
ax[0].set_title("Camera Image")
ax[0].imshow(image)
ax[1].set_title("Env Image")
plt_r = ax[1].imshow(render)
plt_d = ax[2].imshow(np.linalg.norm(image-render,axis=2))
plt.show()


cal_vars = OrderedDict(
    rbt_z    = (0.,3,.01,0.5),
    obj_pos_x=(-.2,.2,.01,.14),
    obj_pos_y=(-.2,.2,.01,.13),
    obj_orn_r=(-1,1,.1,-.8),
    srf_col_r=(0,1,.1,0.4), srf_col_g=(0,1,.1,0.5), srf_col_b=(0,1,.1,0.2),
    obj_col_r=(0,1,.1,0.2), obj_col_g=(0,1,.1,0.2), obj_col_b=(0,1,.1,0.2),
    grp_col_r=(0,1,.1,0.1), grp_col_g=(0,1,.1,0.1), grp_col_b=(0,1,.1,0.1),
)

def get_image(**argv):
    # robot position
    rbt_z = argv['rbt_z']
    env.robot.apply_action([0.0, 0.0, rbt_z, 0, 1.0],)
    
    # object position
    obj_pos, obj_orn = list(pos_orig),list(orn_orig)
    obj_pos[0] += argv['obj_pos_x']
    obj_pos[1] += argv['obj_pos_y']
    obj_orn = env.p.getQuaternionFromEuler((0,0,argv['obj_orn_r']))
    env.p.resetBasePositionAndOrientation(env._task.objects[0], obj_pos, obj_orn)
    
    # surface color
    assert(len(env._task.surfaces) == 1)
    surface_uid = env._task.surfaces[0]
    surface_col = (argv['srf_col_r'],argv['srf_col_g'],argv['srf_col_b'],1.0)
    #print(argv)
    env._task.set_uid_color(surface_uid,surface_col)
    
    # object color
    object_col = (argv['obj_col_r'],argv['obj_col_g'],argv['obj_col_b'],1.0)
    for obj_uid in env._task.objects:
        env._task.set_uid_color(obj_uid,object_col)
    
    # gripper color
    assert(len(env._task.surfaces) == 1)
    robot_uid = env.robot.robot_uid
    finger_ids = env.robot.gripper.finger_link_ids
    gripper_col = (argv['grp_col_r'],argv['grp_col_g'],argv['grp_col_b'],1.0)
    for link_id in finger_ids:
        env._task.set_uid_color(robot_uid,gripper_col,link_id=link_id)
    
    return env.render()
    
def update(**var_def):
    render = get_image(**var_def)
    plt_r.set_data(render)
    diff = np.linalg.norm(image-render,axis=2)
    print("loss",np.mean(diff))
    plt_d.set_data(diff)
    fig.canvas.draw()
    
var_range = dict([(k,v[0:3]) for k,v in cal_vars.items()])
w = interactive(update, **var_range)
display(w)

for c in w.children:
    try:
        name = c.description
    except AttributeError:
        continue
    if name not in cal_vars:
        continue
    var = cal_vars[name]
    if len(var) == 3:
        continue
    c.value = var[3]
    


In [ ]:
#plt.imshow(default_target)
#plt.show()
minscore = 1e30
def testfunc(arr):
    argv = dict(zip(cal_vars.keys(), arr))
    render = get_image(**argv)
    diff = np.linalg.norm(image-render,axis=2)
    norm = diff.mean()
    global minscore
    if norm < minscore:
        print(minscore, arr)
        minscore = norm
    return(norm)

from scipy.optimize import differential_evolution
from scipy.optimize import minimize

bounds = []
for k,v in cal_vars.items():
    mean = v[3]
    delta = abs(v[1]-v[0])*.1
    bounds.append((mean-delta,mean+delta))

result = differential_evolution(testfunc, bounds)
#result = minimize(testfunc, x0=arr,method="Nelder-Mead")

print("\nOptimization done!")
print(result.x, result.fun)
    

In [ ]:
arr = [ 0.43711184,  0.11892019,  0.15063476, -0.78692746,  0.44709236,  0.51310167,
  0.27719088,  0.20437309,  0.24215557,  0.23598924, 0.15451711,  0.12243651,
  0.11153517]
arr = result.x
argv = dict(zip(cal_vars.keys(), arr))
render = get_image(**argv)

fig, ax = plt.subplots(1,3)
for x in ax: x.set_axis_off()
fig.suptitle("Default Views (different aspect ratio)")
ax[0].set_title("Camera Image")
ax[0].imshow(image)
ax[1].set_title("Env Image")
plt_r = ax[1].imshow(render)
plt_d = ax[2].imshow(np.linalg.norm(image-render,axis=2))
plt.show()

In [ ]:
image_hsv = Image.fromarray(image).convert('HSV')
image_hsv = np.array(image_hsv)[:,:,1]
plt.imshow(image_hsv)
plt.show()

In [ ]:
from skimage import data
from skimage.color import rgb2gray
from skimage.morphology import disk
from skimage.filters.rank import gradient

fig, ax = plt.subplots(1,1)
cat_gray = rgb2gray(image)
selection_element = disk(1) # matrix of n pixels with a disk shape

cat_sharpness = gradient(cat_gray, selection_element)
ax.imshow(cat_sharpness, cmap="viridis")
ax.axis('off')

plt.show()